In [72]:
import pandas as pd
import json
import ast
import sqlite3
import numpy as np
from surprise import SVD
from surprise import NMF
from surprise import KNNBasic
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
## Analysis tools 
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score, make_scorer
from sklearn import model_selection
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors

In [3]:
with sqlite3.connect('steam_data.sqlite') as conn:
    steam_games = pd.read_sql("SELECT * FROM steam_games",conn)
    aus_user_reviews_df = pd.read_sql("SELECT * FROM australian_user_reviews",conn)
    aus_user_items_df = pd.read_sql("SELECT * FROM australian_users_items",conn)
    bundle_df = pd.read_sql("SELECT * FROM bundle_data",conn)

In [54]:
steam_games.describe()

,index,discount_price,early_access
count,32135.0,225.000000,32135.000000
mean,0.0,11.930533,0.060588
std,0.0,17.492643,0.238577
min,0.0,0.490000,0.000000
25%,0.0,1.390000,0.000000
50%,0.0,4.190000,0.000000
75%,0.0,22.660000,0.000000
max,0.0,139.990000,1.000000


In [62]:
#drop_columns = ['index', 'funny', 'last_edited','helpful', 'review', 'user_url','posted']
#aus_user_reviews_df.drop(drop_columns, inplace=True, axis=1)
aus_user_reviews_df

,user_id,item_id,recommend
0,76561197970982480,1250,1
1,76561197970982480,22200,1
2,76561197970982480,43110,1
3,js41637,251610,1
4,js41637,227300,1
5,js41637,239030,1
6,evcentric,248820,1
7,evcentric,370360,1
8,evcentric,237930,1
9,evcentric,263360,1


In [31]:
#aus_user_reviews_df.head(4)

In [7]:
drop_columns = ['index', 'items_count', 'steam_id', 'user_url','item_name', 'playtime_forever','playtime_2weeks']
aus_user_items_df.drop(drop_columns, inplace = True, axis=1)


In [32]:
aus_user_items_df.head(20)

,user_id,item_id
0,76561197970982479,10
1,76561197970982479,20
2,76561197970982479,30
3,76561197970982479,40
4,76561197970982479,50
5,76561197970982479,60
6,76561197970982479,70
7,76561197970982479,130
8,76561197970982479,300
9,76561197970982479,240


In [9]:
# form game genres binary dataframe
user_game_genres = pd.read_sql("select user_id, item_id, genres, discount_price, price from australian_user_reviews left join steam_games on item_id=id where recommend=True and genres is not null and genres <>'null' and specs is not null and specs <>'null' and tags is not null and tags <>'null';", conn)
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(user_game_genres['genres'].apply(eval)), columns=mlb.classes_)
user_game_genres['discount_price'].fillna(0, inplace=True)
user_game_genres['price'].replace({'Play the Demo':0,'Play WARMACHINE: Tactics Demo':0,'Install Now':0,'Free Mod':0,'Third-party':0,'Free To Play': 0, 'Free to Play':0,'free':0,'Free':0, 'Play for Free!':0, 'Free HITMAN™ Holiday Pack':0}, inplace=True)
genres_result = pd.concat([user_game_genres['item_id'], user_game_genres['discount_price'], user_game_genres['price'], encoded], axis=1)

#engine = create_engine('sqlite:///steam_data.sqlite')

In [10]:
# remove duplicates under item_id
genres_result.drop_duplicates(['item_id'], inplace=True)

In [8]:
#genres_result

In [11]:
# price column min-max normalization
x_price = genres_result[['price']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_price_scaled = min_max_scaler.fit_transform(x_price)


genres_result['price'] = pd.DataFrame(x_price_scaled)

In [12]:
# discount_price column min-max normalization
x_discount_price = genres_result[['discount_price']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_discount_price_scaled = min_max_scaler.fit_transform(x_discount_price)

genres_result['discount_price'] = pd.DataFrame(x_discount_price_scaled)

In [12]:
#genres_result

In [13]:
#form game specs binary dataframe
user_game_specs = pd.read_sql("select item_id, specs from australian_user_reviews left join steam_games on item_id=id where recommend=True and specs is not null and genres <>'null';", conn)
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(user_game_genres['genres'].apply(eval)), columns=mlb.classes_)
specs_result = pd.concat([user_game_specs['item_id'], encoded], axis=1)

In [14]:
#remove duplicates under item_id
specs_result.drop_duplicates(['item_id'], inplace=True)

In [15]:
#form game tags binary dataframe
user_game_tags = pd.read_sql("select item_id, tags from australian_user_reviews left join steam_games on item_id=id where recommend=True and tags is not null and tags <>'null';", conn)
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(user_game_tags['tags'].apply(eval)), columns=mlb.classes_)
tags_result = pd.concat([user_game_specs['item_id'], encoded], axis=1)

In [16]:
#remove duplicates under item_id
tags_result.drop_duplicates(['item_id'], inplace=True)

In [17]:
item_df = pd.merge(genres_result, specs_result, on = 'item_id')
item_df = pd.merge(item_df, tags_result, on = 'item_id')

In [69]:
#item_df

In [18]:
item_df.fillna(0, inplace=True)


In [29]:
#item_df[item_df['item_id']=='100']

In [22]:
#Item_DF = pd.merge(aus_user_items_df, item_df, on='item_id' )

In [40]:
# item_list including all items
item_list=[]
for item in item_df['item_id']:
    item_list.append(item)
    
# based on user's purchase record to find the 5 most similar items in the unpurchased item list  
def item_based_recommendation(user_id, number=5):
    number = number
    user_item_purchased_df = aus_user_items_df[aus_user_items_df['user_id']== user_id].reset_index(drop=True)
    user_item_purchased_df = pd.merge(user_item_purchased_df, item_df, on = 'item_id', how = 'left')
    user_item_purchased_df.dropna(inplace = True)
    user_item_purchased_list=[]
    user_item_unpurchased_list=[]
    user_item_recommend_list=[]
    for item in user_item_purchased_df['item_id']:
        user_item_purchased_list.append(item)
    for item in item_list:
        if(item not in user_item_purchased_list):
            user_item_unpurchased_list.append(item)
    user_item_unpurchased_df = pd.DataFrame()
    user_item_unpurchased_df['item_id'] = user_item_unpurchased_list
    # form a DataFrame including user's unpurchased items
    user_item_unpurchased_df = pd.merge(user_item_unpurchased_df, item_df, on = 'item_id', how = 'left')
    # only look for 1 nearest neighbor
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(user_item_unpurchased_df.drop('item_id',axis=1)) 
    user_columns = ['user_id', 'item_id']
    user_item_neighbor_list = neigh.kneighbors(user_item_purchased_df.drop(user_columns, axis=1), return_distance=False)
    if(len(user_item_neighbor_list)>number):
        for item in user_item_neighbor_list[:number]:
            user_item_recommend_list.append(user_item_unpurchased_df['item_id'].iloc[item])
    else:
        for item in user_item_neighbor_list:
            user_item_recommend_list.append(user_item_unpurchased_df['item_id'].iloc[item])
    return user_item_recommend_list
    #(user_item_recommend_list)
    

In [60]:
a = pd.DataFrame(item_based_similarity('76561197970982479', number=10)).reset_index(drop=True)


In [61]:
a

,397,539,1335,1468,1501,1753,1905,1970,2185,2270
0,NaN,NaN,NaN,NaN,485380,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,313372,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290990,NaN,NaN
3,265300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212410,NaN
5,NaN,NaN,15300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,42719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,300760,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,408900
9,NaN,NaN,NaN,NaN,NaN,NaN,340300,NaN,NaN,NaN


In [70]:
#item_df[item['item_id']=='485380']

In [25]:
item_df[item_df['item_id'] == '265300']

,item_id,discount_price,price,Action_x,Adventure_x,Animation &amp; Modeling_x,Audio Production_x,Casual_x,Design &amp; Illustration_x,Early Access_x,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
506,265300,0.0,0.025904,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
user_df = aus_user_items_df[aus_user_items_df['user_id']=='76561197970982479'].reset_index(drop=True)
user_df = pd.merge(user_df, item_df, on='item_id', how= 'left')
user_df.dropna(inplace=True)
user_item_list=[]
item_list=[]
user_r_list=[]
for item in item_df['item_id']:
    item_list.append(item)
for item in user_df['item_id']:
    user_item_list.append(item)

for item in item_list:
    if(item not in user_item_list):
        user_r_list.append(item)
user_r_df = pd.DataFrame()
user_r_df['item_id'] = user_r_list
user_r_df = pd.merge(user_r_df, item_df, on = 'item_id', how = 'left')

In [65]:
neigh = NearestNeighbors(n_neighbors=1, metric='jaccard')
neigh.fit(user_r_df.drop('item_id',axis=1)) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='jaccard',
                 metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                 radius=1.0)

In [66]:
user_columns = ['user_id', 'item_id']
a = neigh.kneighbors(user_df.drop(user_columns, axis=1), return_distance=False)

In [67]:
a[1]

array([1468])

In [51]:

a_list = []
for i in a:
    pd.concat()

In [53]:
a_list[1]

1314    313372
Name: item_id, dtype: object

In [54]:
item_df[item_df['item_id']=='313372']

,item_id,discount_price,price,Action_x,Adventure_x,Animation &amp; Modeling_x,Audio Production_x,Casual_x,Design &amp; Illustration_x,Early Access_x,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
1647,313372,0.0,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
steam_games
drop_columns = ['index', 'app_name', 'metascore', 'reviews_url','title', 'url']
steam_games_df = steam_games.drop(drop_columns, axis = 1)

In [68]:
#steam_games_df.head()

In [171]:
#steam_games_df['specs'][1].split(',')[2]

In [125]:
new_df_classification = pd.merge(aus_user_reviews_df,steam_games_df, left_on = 'item_id', right_on = 'id' )
#_ = pd.merge(new_df_classification, aus_user_items_df, left_on = ['user_id', 'item_id'], right_on = ['user_id', 'item_id'])

In [126]:
df = pd.merge(new_df_classification, aus_user_items_df, left_on = ['user_id', 'item_id'], right_on = ['user_id', 'item_id'])

In [71]:
#df.head(7)

In [214]:
labels.value_counts()

like       48206
dislike     5782
Name: recommend, dtype: int64

In [212]:
#labels

In [199]:
new_df_classification.drop('recommend', axis=1,inplace=True)

In [229]:
train_columns = [x for x in new_df_classification.columns if x not in ['user_id', 'item_id','developer']]

In [230]:
train = new_df_classification[train_columns]

In [231]:
train.head(2)

,discount_price,genres,price,publisher,sentiment,specs,tags
0,0.0,"[""Action""]",19.99,Tripwire Interactive,Overwhelmingly Positive,"[""Single-player"", ""Multi-player"", ""Co-op"", ""Cr...","[""FPS"", ""Zombies"", ""Co-op"", ""Survival"", ""Actio..."
1,0.0,"[""Action""]",19.99,Tripwire Interactive,Overwhelmingly Positive,"[""Single-player"", ""Multi-player"", ""Co-op"", ""Cr...","[""FPS"", ""Zombies"", ""Co-op"", ""Survival"", ""Actio..."


In [206]:
#new_df_regression.unstack()

In [75]:
reader = Reader(rating_scale=(0, 1))
data_training = Dataset.load_from_df(aus_user_reviews_df[['user_id', 'item_id', 'recommend']], reader)
#data_testing = Dataset.load_from_df(testings[['reviewerID', 'asin', 'overall']], reader)

In [211]:
aus_user_reviews_df['recommend'].value_counts()

1    52473
0     6832
Name: recommend, dtype: int64

In [76]:
trainset, testset = train_test_split(data_training, test_size=.25)

TypeError: Singleton array array(<surprise.dataset.DatasetAutoFolds object at 0x1a801de978>,
      dtype=object) cannot be considered a valid collection.

In [77]:
# SVD Algorithm
algo_svd = SVD(n_factors= 400, reg_all=0.001)
kf = KFold(n_splits=5)

In [78]:
for trainset, testset in kf.split(data_training):

    # train and test algorithm.
    algo_svd.fit(trainset)
    predictions = algo_svd.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.3160
RMSE: 0.3218
RMSE: 0.3215
RMSE: 0.3188
RMSE: 0.3195


In [207]:
predictions[1]

Prediction(uid='EdgyAllenPoe', iid='107100', r_ui=1.0, est=0.9991531630122946, details={'was_impossible': False})

In [82]:
# item_list including all items
item_list=[]
for item in item_df['item_id']:
    item_list.append(item)
def SVD_model_recommendation(user_id, number=5):
    number = number
    user_item_purchased_df = aus_user_items_df[aus_user_items_df['user_id']== user_id].reset_index(drop=True)
    user_item_purchased_df = pd.merge(user_item_purchased_df, item_df, on = 'item_id', how = 'left')
    user_item_purchased_df.dropna(inplace = True)
    user_item_purchased_list=[]
    user_item_unpurchased_list=[]
    user_item_recommend_list=[]
    for item in user_item_purchased_df['item_id']:
        user_item_purchased_list.append(item)
    for item in item_list:
        if(item not in user_item_purchased_list):
            user_item_unpurchased_list.append(item)
    user_item_unpurchased_df = pd.DataFrame()
    user_item_unpurchased_df['item_id'] = user_item_unpurchased_list
    user_item_unpurchased_df['user_id'] = user_id
    user_item_unpurchased_df['recommend'] = np.nan
    data_testing = Dataset.load_from_df(user_item_unpurchased_df[['user_id', 'item_id', 'recommend']], reader)
    data_testingset = data_testing.build_full_trainset().build_testset()
    predictions = algo_svd.test(data_testingset)
    predictions_df = pd.DataFrame(predictions)
    predictions_df.sort_values(by='est', ascending=False, inplace=True)
    
    #if(predictions_df.shape(0)>number):
    return predictions_df['item_id'][:number]
    #else:
        #return predictions_df['item_id']

In [83]:
SVD_model_recommendation('76561197970982479',5)

KeyError: 'item_id'

In [102]:
accuracy.rmse(predictions_knn, verbose=True) 

RMSE: 0.3309


0.33093247018372685

In [208]:
predictions_knn[1]

Prediction(uid='76561198059715440', iid='218060', r_ui=1.0, est=0.8848326448022932, details={'was_impossible': True, 'reason': 'Not enough neighbors.'})

In [5]:
# bundle_data DataFrame
with open('bundle_data.json') as f:
    lines = f.readlines()
    
bundle_list = []
for line in lines:
    line = ast.literal_eval(line)
    bundle_list.append(line)
bundle_df = pd.DataFrame(bundle_list)
#bundle_df

In [50]:
#bundle_df.to_csv('bundle.csv', index=None)
bundle_df.to_json('bundle.json')

In [84]:
#bundle_df.loc[5,'items']

In [31]:
# australian_user_reviews DataFrame
with open('australian_user_reviews.json') as f:
    lines = f.readlines()
    
aus_user_review_list = []
for line in lines:
    line = ast.literal_eval(line)
    aus_user_review_list.append(line)
aus_user_review_df = pd.DataFrame(aus_user_review_list)


In [85]:
#aus_user_review_df

In [5]:
user_list=[]
item_list=[]
recommend_list=[]
for user in aus_user_review_df['user_id']:
    for item in aus_user_review_df['reviews']:
        for i in item:
            user_list.append(user)
            item_list.append(i["item_id"])
            recommend_list.append(i["recommend"])

In [86]:
#as_df = pd.DataFrame()
#as_df['recommend'] = recommend_list

In [ ]:
as_df.to_csv('recommend.csv', index=None)

In [ ]:
as_df = pd.DataFrame()
as_df['user_id'] = user_list
as_df['item_id'] = item_list
as_df['recommend'] = recommend_list

In [ ]:
as_df.to_csv('austrilian_user_reviews_c.csv')

In [6]:
# steams_data DataFrame
with open('steam_games.json') as f:
    lines = f.readlines()
    #df_steam = pd.DataFrame(lines)  
steam_list = []
for line in lines:
    line = ast.literal_eval(line)
    steam_list.append(line)
steam_df = pd.DataFrame(steam_list)

In [87]:
#steam_df.head(11)

In [9]:
type(steam_df.loc[0,'early_access'])

numpy.bool_

In [71]:
steam_df.to_json('steam_games_c.json')

In [72]:
steam_df_c = pd.read_json('steam_games_c.json')

In [80]:
type(steam_df_c.iloc[1, 5])

numpy.float64

In [33]:
# austrilians user reviews DataFrame
with open('australian_user_reviews.json') as f:
    lines = f.readlines()
user_review_list = []
for line in lines:
    line = ast.literal_eval(line)
    user_review_list.append(line)
user_review_df = pd.DataFrame(user_review_list)

In [34]:
user_review_df.loc[0, 'reviews'][1]['item_id']


'22200'

In [35]:
# austrilians users items DataFrame
with open('australian_users_items.json') as f:
    lines = f.readlines()
    #df_u_i = pd.DataFrame(lines)  
user_items_list = []
for line in lines:
    line = ast.literal_eval(line)
    user_items_list.append(line)
user_items_df = pd.DataFrame(user_items_list)

In [36]:
#user_items_df.head(5)

,items,items_count,steam_id,user_id,user_url
0,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
2,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137,76561198007712555,evcentric,http://steamcommunity.com/id/evcentric
3,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328,76561197963445855,Riot-Punch,http://steamcommunity.com/id/Riot-Punch
4,"[{'item_id': '300', 'item_name': 'Day of Defea...",541,76561198002099482,doctr,http://steamcommunity.com/id/doctr


In [88]:
#user_items_df.loc[1, 'items']

In [ ]:
#chunck = pd.read_json('steam_reviews.json', lines=True,)

In [89]:
#user_items_df.to_json('user_items.json')

In [68]:
df = pd.read_json('user_items.json')

In [70]:
df.loc[10000, 'items']

[]

In [51]:
df= pd.read_json('bundle.json')

In [57]:
df.loc[0,'items'][1]['item_id']

'331490'

In [158]:
user_game_genres = pd.read_sql("select user_id, item_id, genres, discount_price, price from australian_user_reviews left join steam_games on item_id=id where recommend=True and genres is not null and genres <>'null' and specs is not null and specs <>'null' and tags is not null and tags <>'null';", conn)
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(user_game_genres['genres'].apply(eval)), columns=mlb.classes_)
user_game_genres['discount_price'].fillna(0, inplace=True)
user_game_genres['price'].replace({'Play the Demo':0,'Play WARMACHINE: Tactics Demo':0,'Install Now':0,'Free Mod':0,'Third-party':0,'Free To Play': 0, 'Free to Play':0,'free':0,'Free':0, 'Play for Free!':0, 'Free HITMAN™ Holiday Pack':0}, inplace=True)
result = pd.concat([user_game_genres['user_id'], user_game_genres['item_id'], user_game_genres['discount_price'], user_game_genres['price'], encoded], axis=1)

#engine = create_engine('sqlite:///steam_data.sqlite')

In [159]:
result.drop('user_id', axis =1, inplace=True)
result.drop_duplicates(['item_id'], inplace=True)

In [161]:
# price column min-max normalization
# Create x, where x the 'scores' column's values as floats
x_price = result[['price']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x_price)

# Run the normalizer on the dataframe
#df_normalized = pd.DataFrame(x_scaled)
result['price'] = pd.DataFrame(x_scaled)

In [163]:
# discount_price column min-max normalization
# Create x, where x the 'scores' column's values as floats
x_discount_price = result[['discount_price']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x_discount_price)

# Run the normalizer on the dataframe
#df_normalized = pd.DataFrame(x_scaled)
result['discount_price'] = pd.DataFrame(x_scaled)

In [167]:
#result

,item_id,discount_price,price,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,Massively Multiplayer,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,1250,0.0,0.025904,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22200,0.0,0.012945,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,227300,0.0,0.025904,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,239030,0.0,0.012945,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,248820,0.0,0.012945,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,370360,0.0,0.009058,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,237930,0.0,0.025904,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,263360,0.0,0.009058,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,107200,0.0,0.012945,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
9,224500,0.0,0.019424,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [68]:
#user_game_specs = pd.DataFrame()
#user_game_specs['id'] = steam_games_df['id']
#user_game_specs['specs'] = steam_games_df['specs']
#user_game_specs.dropna(inplace = True)
#user_game_specs['specs'].replace('null', np.nan, inplace=True)
#user_game_specs.dropna(inplace = True)

In [77]:
specs_encoded = pd.DataFrame(mlb.fit_transform(user_game_specs['specs'].apply(eval)), columns=mlb.classes_)
specs_result = pd.concat([user_game_specs['id'], specs_encoded], axis=1)
#specs_result.drop_duplicates(['id'])
#user_game_specs

In [78]:
#specs_result

In [91]:
user_game_specs = pd.read_sql("select user_id, item_id, specs from australian_user_reviews left join steam_games on item_id=id where recommend=True and specs is not null and genres <>'null';", conn)
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(user_game_genres['genres'].apply(eval)), columns=mlb.classes_)
specs_result = pd.concat([user_game_specs['user_id'], user_game_specs['item_id'], encoded], axis=1)



In [92]:
specs_result.drop('user_id', axis =1, inplace=True)
specs_result.drop_duplicates(['item_id'], inplace=True)

In [93]:
specs_result

,item_id,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Massively Multiplayer,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,1250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,227300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,239030,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,248820,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,370360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,237930,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,263360,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,107200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,224500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [94]:
user_game_tags = pd.read_sql("select user_id, item_id, tags from australian_user_reviews left join steam_games on item_id=id where recommend=True and tags is not null and tags <>'null';", conn)
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(user_game_tags['tags'].apply(eval)), columns=mlb.classes_)
tags_result = pd.concat([user_game_specs['user_id'], user_game_specs['item_id'], encoded], axis=1)

In [95]:
tags_result.drop('user_id', axis =1, inplace=True)
tags_result.drop_duplicates(['item_id'], inplace=True)

In [124]:
#tags_result

In [168]:
item_df = pd.merge(result, specs_result, on = 'item_id')
item_df = pd.merge(item_df, tags_result, on = 'item_id')

In [178]:
item_df.fillna(0, inplace=True)

In [179]:
item_df

,item_id,discount_price,price,Action_x,Adventure_x,Animation &amp; Modeling_x,Audio Production_x,Casual_x,Design &amp; Illustration_x,Early Access_x,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,1250,0.0,0.025904,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,22200,0.0,0.012945,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,227300,0.0,0.025904,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,239030,0.0,0.012945,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,248820,0.0,0.012945,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,370360,0.0,0.009058,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,237930,0.0,0.025904,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,263360,0.0,0.009058,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,107200,0.0,0.012945,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,224500,0.0,0.019424,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
